In [1]:
import numpy as np 
import os

n_modes=4
run=2

# BASE_DIR = "//sum-lpnc-nas.u-ga.fr/SecureVault/LPNC-SecureVault/MEGAGING/Processed/osl_processing"
BASE_DIR = "/run/user/1001/gvfs/smb-share:server=sum-lpnc-nas.u-ga.fr,share=securevault/LPNC-SecureVault/MEGAGING/Processed/osl_processing"

MODEL_DIR = f"{BASE_DIR}/train_dynemo_1_90/{n_modes:02d}_modes/run{run:02d}/model"
OUTPUT_DIR = f"{BASE_DIR}/train_dynemo_1_90/{n_modes:02d}_modes/run{run:02d}/inf_params"
os.makedirs(f"{OUTPUT_DIR}", exist_ok=True)

# Demographics & Covariates
age_group_dict = {
    'bm_014': 1, # young
    'ca_001': 1,
    'ca_019': 2, # old
    'cc_007': 2,
    'cm_013': 2,
    'dm_022': 1,
    'el_018': 1,
    'gb_020': 2,
    'gh_017': 1,
    'gp_011': 2,
    'gv_005': 2,
    'lf_012': 2,
    'lr_008': 1,
    'pe_009': 1,
    'pl_016': 1,
    'pr_015': 2,
    'ra_003': 1,
    're_002': 1,
    'sg_010': 1
}
age = np.array(list(age_group_dict.values()))

In [4]:
f = np.load(f"{OUTPUT_DIR}/f.npy")
psd = np.load(f"{OUTPUT_DIR}/psd.npy")
coh = np.load(f"{OUTPUT_DIR}/coh.npy")
w = np.load(f"{OUTPUT_DIR}/w.npy")

from osl_dynamics.analysis import power, connectivity
psd_coefs = psd[:, 0]
pow = power.variance_from_spectra(f,psd_coefs)
print(pow.shape)

coh = connectivity.mean_coherence_from_spectra(f,coh)
m, n = np.triu_indices(coh.shape[-1],k=1)
coh = coh[...,m,n]
print(coh.shape)

(19, 4, 52)
(19, 4, 1326)


In [8]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.model_selection import KFold, GridSearchCV
from sklearn.linear_model import ElasticNet

ElasticNet(max_iter=10000)

X = np.concatenate([pow,coh],axis=-1)
X = X.reshape(X.shape[0],-1)
print(X.shape)

Y = age
print(Y.shape)

(19, 5512)
(19,)


In [16]:
kf = KFold(n_splits=5,shuffle=True,random_state=0)
pipeline = Pipeline(
    [
        ("scaler", StandardScaler()),
        ("pca", PCA(whiten=True)),
        ("reg", ElasticNet())
    ]
)
param_grid = {
    "pca__n_components": range(2,10,2),
    "reg__alpha": np.logspace(-3,3,7),
    "reg__l1_ratio": [0.1,0.5,0.9]
}

scores=[]
for fold, (train_indices, test_indices) in enumerate(kf.split(Y)):
    X_train, X_test, Y_train, Y_test = X[train_indices], X[test_indices], Y[train_indices], Y[test_indices]
    reg = GridSearchCV(pipeline, param_grid,n_jobs=16)
    reg.fit(X_train, Y_train)

    score = reg.score(X_test, Y_test)
    scores.append(score)

    print(f"Fold {fold}: best_params={reg.best_params_} R2={score}")

print()
print(f"Mean R2: {np.mean(scores)}")

Fold 0: best_params={'pca__n_components': 4, 'reg__alpha': 0.001, 'reg__l1_ratio': 0.1} R2=-0.1811676899071888
Fold 1: best_params={'pca__n_components': 6, 'reg__alpha': 0.1, 'reg__l1_ratio': 0.1} R2=-0.1395760841315905
Fold 2: best_params={'pca__n_components': 2, 'reg__alpha': 1.0, 'reg__l1_ratio': 0.5} R2=-0.0400000190734886
Fold 3: best_params={'pca__n_components': 2, 'reg__alpha': 1.0, 'reg__l1_ratio': 0.5} R2=-0.9259257493195794
Fold 4: best_params={'pca__n_components': 2, 'reg__alpha': 1.0, 'reg__l1_ratio': 0.5} R2=-0.048828125

Mean R2: -0.26709953348636944
